In [2]:
from __future__ import unicode_literals, print_function
from collections import Counter, defaultdict
import itertools
import os, sys, email, re

import numpy as np
import pandas as pd
import spacy
import plac

In [6]:
emails_df = pd.read_csv('emails.csv')

## Helper functions
def get_text_from_email(msg):
    '''To get the content from email objects'''
    parts = []
    for part in msg.walk():
        if part.get_content_type() == 'text/plain':
            parts.append( part.get_payload() )
    return ''.join(parts)

def split_email_addresses(line):
    '''To separate multiple email addresses'''
    if line:
        addrs = line.split(',')
        addrs = frozenset(map(lambda x: x.strip(), addrs))
    else:
        addrs = None
    return addrs

# Parse the emails into a list email objects
messages = list(map(email.message_from_string, emails_df['message']))
emails_df.drop('message', axis=1, inplace=True)
# Get fields from parsed email objects
keys = messages[0].keys()
for key in keys:
    emails_df[key] = [doc[key] for doc in messages]
# Parse content from emails
emails_df['content'] = list(map(get_text_from_email, messages))
# Split multiple email addresses
emails_df['From'] = emails_df['From'].map(split_email_addresses)
emails_df['To'] = emails_df['To'].map(split_email_addresses)

# Extract the root of 'file' as 'user'
emails_df['user'] = emails_df['file'].map(lambda x:x.split('/')[0])
emails_df = emails_df.set_index('Message-ID')\
    .drop(['file', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding'], axis=1)

# Parse datetimeq
emails_df['Date'] = pd.to_datetime(emails_df['Date'], infer_datetime_format=True)
del messages

In [8]:
def filter_spans(spans):
    # Filter a sequence of spans so they don't contain overlaps
    get_sort_key = lambda span: (span.end - span.start, span.start)
    sorted_spans = sorted(spans, key=get_sort_key, reverse=True)
    result = []
    seen_tokens = set()
    for span in sorted_spans:
        if span.start not in seen_tokens and span.end - 1 not in seen_tokens:
            result.append(span)
            seen_tokens.update(range(span.start, span.end))
    return result


def extract_currency_relations(doc):
    # Merge entities and noun chunks into one token
    seen_tokens = set()
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for money in filter(lambda w: w.ent_type_ == "MONEY", doc):
        if money.dep_ in ("attr", "dobj"):
            subject = [w for w in money.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, money))
        elif money.dep_ == "pobj" and money.head.dep_ == "prep":
            relations.append((money.head.head, money))
    return relations

def extract_people_relations(doc):
    # Merge entities and noun chunks into one token
    seen_tokens = set()
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for person in filter(lambda w: w.ent_type_ == "PERSON", doc):
        if person.dep_ in ("attr", "dobj"):
            subject = [w for w in person.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, person))
        elif person.dep_ == "pobj" and person.head.dep_ == "prep":
            relations.append((person.head.head, person))
    return relations

def extract_datetime_relations(doc):
    # Merge entities and noun chunks into one token
    seen_tokens = set()
    spans = list(doc.ents) + list(doc.noun_chunks)
    spans = filter_spans(spans)
    with doc.retokenize() as retokenizer:
        for span in spans:
            retokenizer.merge(span)

    relations = []
    for datetime in filter(lambda w: w.ent_type_ in ['DATE', 'TIME'], doc):
        if datetime.dep_ in ("attr", "dobj"):
            subject = [w for w in datetime.head.lefts if w.dep_ == "nsubj"]
            if subject:
                subject = subject[0]
                relations.append((subject, datetime))
        elif datetime.dep_ == "pobj" and datetime.head.dep_ == "prep":
            relations.append((datetime.head.head, datetime))
    return relations

In [12]:
model = "en_core_web_sm"

nlp = spacy.load(model)
print("Loaded model '%s'" % model)
print("Processing %d emails" % len(emails_df['content']))

for text in emails_df['content']:
    doc = nlp(text)
    relations = extract_currency_relations(doc)
    for r1, r2 in relations:
        print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))

Loaded model 'en_core_web_sm'
Processing 517401 texts
selling   	MONEY	2.50
the rents 	MONEY	roughly $1.25
price     	MONEY	680,000
a land loan	MONEY	476,000
estimated
monthly interest payments	MONEY	3,966
which     	MONEY	at least $20,000
high      	MONEY	25,000
a loan amount	MONEY	15,000
produce   	MONEY	20,000
costs     	MONEY	9,000
the land cost	MONEY	8,000
total improved lot cost	MONEY	19,000
which     	MONEY	$0 to $4,000
the additional equity	MONEY	0-$184,000
the total
lot improvement cost	MONEY	31,000
estimate  	MONEY	10,000
a loan value	MONEY	7,000
the
loan value	MONEY	15,000
a total loan	MONEY	992,000
total cost	MONEY	1,232,645
required  	MONEY	241,000
price     	MONEY	680,000
a land loan	MONEY	476,000
estimated
monthly interest payments	MONEY	3,966
which     	MONEY	at least $20,000
high      	MONEY	25,000
a loan amount	MONEY	15,000
produce   	MONEY	20,000
costs     	MONEY	9,000
the land cost	MONEY	8,000
total improved lot cost	MONEY	19,000
which     	MONEY	$0 to $4,000
the ad

a loan value	MONEY	7,000
the
loan value	MONEY	15,000
a total loan	MONEY	992,000
total cost	MONEY	1,232,645
required  	MONEY	241,000
price     	MONEY	680,000
a land loan	MONEY	476,000
estimated
monthly interest payments	MONEY	3,966
which     	MONEY	at least $20,000
high      	MONEY	25,000
a loan amount	MONEY	15,000
produce   	MONEY	20,000
costs     	MONEY	9,000
the land cost	MONEY	8,000
total improved lot cost	MONEY	19,000
which     	MONEY	$0 to $4,000
the additional equity	MONEY	0-$184,000
the total
lot improvement cost	MONEY	31,000
estimate  	MONEY	10,000
a loan value	MONEY	7,000
the
loan value	MONEY	15,000
a total loan	MONEY	992,000
total cost	MONEY	1,232,645
required  	MONEY	241,000
a price   	MONEY	680,000
shows     	MONEY	700,000
a price   	MONEY	680,000
shows     	MONEY	700,000
cbpres@austin.rr.com	MONEY	09/08/2000 05:35:20 AM
$73.80 psf	MONEY	182,000
his
sales price	MONEY	187,000
sell      	MONEY	203,108
a price   	MONEY	680,000
shows     	MONEY	700,000
sold      	MONEY	around $

harge	MONEY	US$793 million
3D=3D=3D=3D=3D=3D=3D=3D=3D Dividends	MONEY	0.01875
a=
n applicable income tax recovery	MONEY	12
al        	MONEY	1,642
s         	MONEY	631
Clarify Inc. Stock option compensation         from ac=
quisitions	MONEY	32
One-time gains	MONEY	45
one-time charges	MONEY	1,024
a total   	MONEY	1,222
taxes     	MONEY	9
applicable income tax recoveries	MONEY	723
A   - Excludes a total	MONEY	23,242
d         divestitures	MONEY	91
charges   	MONEY	15,054
intangi=
ble assets	MONEY	12,486
costs     	MONEY	2,463
a total   	MONEY	3,575
receivable (less provisions	MONEY	684
revenues  	MONEY	28
a service
mark	MONEY	###
connect   	MONEY	less than $20=
tripled   	MONEY	$25 billion
an average	MONEY	$1.20 per million
compared  	MONEY	4.30
Dynegy    	MONEY	$8 billion
$8 billion	MONEY	1.0 		$1.5 billion
price     	MONEY	1,495
Introductory price	MONEY	1,495
a savings 	MONEY	500
e Net Income Basic	MONEY	$0.22 $
d quarter net income	MONEY	$298 million
down      	MONEY	$737 million
Dilut

Dow Jones)--The California Independent System Operator	MONEY	about $10 million
the ISO   	MONEY	about $900
declare   	MONEY	1,100
a debt    	MONEY	more than $6 billion
grow      	MONEY	nearly $1 billion
Neil Moreno	MONEY	120
Gilbert   	MONEY	#27-is
send      	MONEY	250
get       	MONEY	333
recommend 	MONEY	25.00
fell      	MONEY	430.00-$475.00
fell      	MONEY	200.00-$350.00
power     	MONEY	325.00-$335.00
an average	MONEY	12.75
ranging   	MONEY	62.00-$76.00
an average	MONEY	1.25
an average	MONEY	2.44
A delivery point	MONEY	6.13
a Bloomberg index price	MONEY	70.33
sold      	MONEY	90.00-
dropped   	MONEY	67.90
ranging   	MONEY	75.00
opened    	MONEY	59.00
sold      	MONEY	60.00-$75.00
power     	MONEY	18.17
sold      	MONEY	65.00-$100.00
sold      	MONEY	7.80-$8.30
sold      	MONEY	7.65-$8.15
offered   	MONEY	60.00
down      	MONEY	90.00
selling   	MONEY	75.00-$125.00
declined  	MONEY	60
an average	MONEY	23.95
fell      	MONEY	70.25
noted     	MONEY	32.00
sold      	MONEY	78-$100
sank 

Their revised price target	MONEY	$68/share
the balance	MONEY	36,719
the payoff amount	MONEY	36,304
charged   	MONEY	$$/MMbtu
$3,642,100

Net value	MONEY	89,566
which     	MONEY	165.00
a monthly employee cost	MONEY	94.00
The payoff	MONEY	36,248
The payoff	MONEY	158
The payoff	MONEY	36,248
The payoff	MONEY	158
the 200,000/d leg	MONEY	16,918,907
variable costs	MONEY	0.15
the mid-market spread	MONEY	0.45
the NOI   	MONEY	1,152,695
worked    	MONEY	20,000
The quote 	MONEY	over $26,000
a budget  	MONEY	85-$90/sf
spray     	MONEY	1.30/sf
That      	MONEY	approximately $10,000
compared  	MONEY	over $20,000 additional costs
the NOI   	MONEY	around $300,000
a price   	MONEY	2,700,000
closing   	MONEY	10,000
played    	MONEY	27,740
closing   	MONEY	10,000
the NOI   	MONEY	280,000
up        	MONEY	3,000/unit
an NOI    	MONEY	240,000
The asking price	MONEY	2,700,000
the seller's NOI	MONEY	280,000
The cost  	MONEY	33,750
offered   	MONEY	$2.5 million
sold      	MONEY	$2.7 million
I         	MONEY	3,

...

The airfare costs	MONEY	300-$500
The airfare 
costs	MONEY	300-$500
the package	MONEY	around $700-900/



start     	MONEY	$100 auction dollars
the team  	MONEY	12
the 
team 	MONEY	1
a total   	MONEY	110
Bass2@COMPAQ.com	MONEY	08/18/2000 09:35:58 AM
I         	MONEY	5
I         	MONEY	5
I         	MONEY	5
a check   	MONEY	630
I         	MONEY	5
Bass2@COMPAQ.com	MONEY	08/16/2000 09:53:54 AM
that      	MONEY	$220 million
it        	MONEY	10
the Wellhead Potfolio	MONEY	approximately $275,000
Bid       	MONEY	105,000
=                         $145,000     Bid	MONEY	100,000
Bid       	MONEY	106,000
estimated 	MONEY	170,000
Passes    	MONEY	50
a new fundraising record	MONEY	over $3 million
the top fund-raising team	MONEY	350,000
>         	MONEY	.03/share
sells     	MONEY	14.95
the Wellhead Potfolio	MONEY	approximately $275,000
Bid       	MONEY	105,000
=                         $145,000     Bid	MONEY	100,000
Bid       	MONEY	106,000
estimated 	MONEY	170,000
the Wellhead Potfolio	MONEY	approxi

climbed   	MONEY	over $28
rose      	MONEY	0.50
boosted   	MONEY	$4.4 million
Rex's market cap	MONEY	about $180 million
Buying    	MONEY	0.66
more      	MONEY	$900 million
payment   	MONEY	$965 million
is        	MONEY	65
increased 	MONEY	5
our tournament	MONEY	ONE MILLION DOLLARS
Hunters   	MONEY	5
ran       	MONEY	200
this stock	MONEY	200
buy       	MONEY	8.76
U.S. companies	MONEY	billions of dollars
Your annual subscription	MONEY	59
it        	MONEY	4.80
up        	MONEY	1,000
sold      	MONEY	$1.5 million
the camera people	MONEY	as much as $65
Your annual subscription	MONEY	59
You       	MONEY	up to 100 HIpoints
the yearly maintenance costs	MONEY	just $825,000
The total estimated cost	MONEY	$12.8 million
The city  	MONEY	$2 million
the issuance	MONEY	$9.4 million
The
original bond amount	MONEY	$7.6 million
raised    	MONEY	$9.4 million
the center	MONEY	850,000
Its forecast	MONEY	825,000
Their salaries	MONEY	375,000
an estimated yearly cost	MONEY	$450 to $275
The
payments	MONEY	750,

Their current ten year plan	MONEY	1billion
Your annual subscription	MONEY	59
every order	MONEY	100
$5 on
orders	MONEY	100!</b></font><b><font
provide   	MONEY	an additional $8.2 billion
provide=  	MONEY	an additional $8.2 billion
the average person	MONEY	4,000
Our 1,316% gains	MONEY	25,000
turned    	MONEY	more than $350,000
The company	MONEY	about 64 cents
our guaranteed low price	MONEY	15.00
09Conferenceregistration	MONEY	only $350
rate      	MONEY	only $169
895       	MONEY	1095
the low corporate member=
ship price	MONEY	550
an individual membership price	MONEY	350
subscribing	MONEY	39.99
Roaming charges	MONEY	0.60
valued    	MONEY	3,500
Three Promising New Economy Stocks	MONEY	10
the merger	MONEY	$1.8 billion
estimates 	MONEY	2.1 billion
barrels
beat      	MONEY	a penny
EDS       	MONEY	0.44
a profit  	MONEY	$334 million
this year's EPS estimate	MONEY	2.66
Three Promising New Economy Stocks	MONEY	10
rocketed  	MONEY	0.03
The
firm  	MONEY	$8 million
a market cap	MONEY	just $30 milli

A 3-course menu	MONEY	45
sample menu	MONEY	52
17 people 	MONEY	45
access    	MONEY	103853
saved     	MONEY	500,000
FARE      	MONEY	2061.66
positive revenue	MONEY	90,300
2000 forecast	MONEY	$5.1 million
The current mark	MONEY	approximately $4.7 million loss
shifts    	MONEY	more than $10
shifts    	MONEY	more than $10
shifts    	MONEY	more than $10
expiring  	MONEY	12/22/00  
THROUGH 1/5/01

Aggregate VaR Limit	MONEY	$140MM - $40MM
your stock	MONEY	100
down      	MONEY	$1.35MM VaR
expiring  	MONEY	12/22/00  
THROUGH 1/5/01

Aggregate VaR Limit	MONEY	$140MM - $40MM
this      	MONEY	1
this      	MONEY	1
The daily rate	MONEY	6000
The cost  	MONEY	500
exceed    	MONEY	25,000.00
2000 forecast	MONEY	$5.1 million
all       	MONEY	17MM
we        	MONEY	alternative #1
Interns   	MONEY	3,000
entertainment	MONEY	300
Martin Santora	MONEY	09/25/2000 05:29:39 PM
senior consultants	MONEY	2,000
are       	MONEY	1,750
verify    	MONEY	56,000
a total   	MONEY	10.2MM
a loss    	MONEY	about $20 million 
(

KeyboardInterrupt: 

In [13]:
for text in emails_df['content']:
    doc = nlp(text)
    relations = extract_people_relations(doc)
    for r1, r2 in relations:
        print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))

My suggestion	PERSON	Austin
triangle  	PERSON	Save
the first bullet	PERSON	Allan's list
the meeting	PERSON	Keith
the first bullet	PERSON	Allan's list
the meeting	PERSON	Keith
the package	PERSON	San Marcos
the complete list	PERSON	Enron Online
the complete list	PERSON	Enron Online
fees      	PERSON	Creekside Builders
contract  	PERSON	Creekside Builders
I         	PERSON	Keith
fees      	PERSON	Creekside Builders
contract  	PERSON	Creekside Builders
I         	PERSON	Keith
sent      	PERSON	Lotus Notes
The meeting	PERSON	Richard Burchfield/HOU/ECT
relay     	PERSON	Mike Grigsby
the relationship	PERSON	Creekside Builders
sales numbers	PERSON	Reagan
approval  	PERSON	Brenda Key Stone
Rangel    	PERSON	Phillip
200 longs 	PERSON	Socal
the correlation	PERSON	Nymex
you       	PERSON	Debbie Nowak
spoke     	PERSON	Reagan
a meeting 	PERSON	Jean Mrha
11:41 AM  	PERSON	John D Suarez/HOU/ECT@ECT
opened    	PERSON	Tukwila
you       	PERSON	Tori K.'s and Martin Cuilla's resumes and past performance 

relay     	PERSON	Mike Grigsby
the relationship	PERSON	Creekside Builders
sales numbers	PERSON	Reagan
approval  	PERSON	Brenda Key Stone
200 longs 	PERSON	Socal
the correlation	PERSON	Nymex
you       	PERSON	Debbie Nowak
spoke     	PERSON	Reagan
a meeting 	PERSON	Jean Mrha
11:41 AM  	PERSON	John D Suarez/HOU/ECT@ECT
opened    	PERSON	Tukwila
you       	PERSON	Tori K.'s and Martin Cuilla's resumes and past performance 
reviews
05:39 AM  	PERSON	Phillip K Allen/HOU/ECT@ECT
I         	PERSON	Mike
The title 	PERSON	Mark's Page
Colleen   	PERSON	Mike Grigsby
regard    	PERSON	Tori Kuykendall
a similiar situation	PERSON	Tori
same      	PERSON	Tori
Outperform         
		
		  10/25/2000   
		  Lehman Brothers   
		     Downgraded	PERSON	Outperform 
		
the leading Internet provider	PERSON	U.S. Stock
Stephanie

Please	PERSON	the requested eol rights
I         	PERSON	Zimin Lu
Today's best links	PERSON	Al Gore
From      	PERSON	Phillip K Allen/HOU/ECT@ECT
Hunter    	PERSON	Alberto
you       	PERS

Hearts    	PERSON	Atlantis   
Volume    	PERSON	Volume Hedged
Downgraded	PERSON	Hold  
d         	PERSON	Nt Accum
aded      	PERSON	Buy  
sounds    	PERSON	Stephen Bergstrom
the west desk calendar	PERSON	Ina
any early afternoon departures	PERSON	Ina
Rebecca W.  
Sent	PERSON	Lawner
Rebecca W.  
Sent	PERSON	Allen
Downgraded	PERSON	Hold  
aded      	PERSON	Strong Buy
a structure	PERSON	Nolan Goplin
Smith Barney  Downgraded	PERSON	Neutral  
Downgraded	PERSON	Hold  
Wachovia Securities  Downgraded	PERSON	Neutral  
Downgraded	PERSON	Hold  
Downgraded	PERSON	Hold  
ded       	PERSON	Nt Neutral  
ded       	PERSON	Nt Accum    
Downgraded	PERSON	Hold  
Downgraded	PERSON	Nt Accum  
Downgraded	PERSON	Add  
ded       	PERSON	Buy  
see       	PERSON	Davenport
see       	PERSON	Dorothy Barnes
Marc  
Sent	PERSON	Cooper
Open system	PERSON	Max Oper
leaving   	PERSON	Socal Hotline
assume    	PERSON	OFO
Customer  	PERSON	GIMB charge
Phillip K.  
Sent	PERSON	Bhatia
Downgraded	PERSON	Hold  
Upgraded  	PERS

KeyboardInterrupt: 

In [20]:
for text in emails_df['content']:
    doc = nlp(text)
    relations = extract_datetime_relations(doc)
    for r1, r2 in relations:
        print("{:<10}\t{}\t{}".format(r1.text, r2.ent_type_, r2.text))

shoot     	DATE	Tuesday
shoot     	TIME	11:45
buck.buckner@honeywell.com	TIME	10/12/2000 01:12:21 PM
consumed  	DATE	May through September
the meeting	DATE	Wednesday
the meeting	DATE	Wednesday
happen    	DATE	Monday
shown     	DATE	monthly positions
Gas Daily 	DATE	monthly index
the meeting	DATE	Wednesday
the meeting	DATE	Wednesday
happen    	DATE	Monday
shown     	DATE	monthly positions
Gas Daily 	DATE	monthly index
cbpres@austin.rr.com	TIME	10/03/2000 06:35:56 AM
Forwarded 	TIME	04:13 PM
a meeting 	DATE	Tuesday, Oct. 10th
entered   	DATE	September 26 and 27
are       	DATE	those days
the 
marketplace	DATE	September 26 and 27
are       	DATE	those days
represent 	DATE	up to 4 months
represent 	DATE	up to 4 months
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
moved     	TIME	4:00

Phillip K Allen/HOU/ECT	TIME	04:01 PM
meeting   	DATE	Sunday
<cbpres@austin.rr.com	TIME	03/13/2001 11:29:49 PM
invested  	DATE	more than five months
Friday    	DATE	this week
stephen.benotti@painewebber.com	TIME	03/13/2001 12:58:24 PM
Database  	TIME	hourly storage activity
a rentroll	DATE	this week
know      	DATE	about two weeks
coming    	DATE	the next 12 months
schedule  	TIME	1 p.m.
be        	DATE	Tuesday
07:10 AM  	TIME	02/28/2001 02:14:43 PM
Only Enron's deals	DATE	yesterday
as        	DATE	2/27
paid      	DATE	February
is        	DATE	the beginning of 
the year
I         	DATE	a day
take      	DATE	the week
02:27 PM  	TIME	02/21/2001 07:28:47 AM
New Generation Report	DATE	January 2001
"George Richards	TIME	02/21/2001 06:32:15 AM
"George Richards	TIME	02/21/2001 01:10:33 PM
reschedule	DATE	tomorrow
meet      	DATE	Monday
close     	DATE	April
.         	DATE	this week
a single weekly call	DATE	Monday
"George Richards	TIME	02/21/2001 06:32:15 AM
John      	TIME	3pm
Sarah Novosel

represent 	DATE	up to 4 months
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
occurs    	DATE	a weekend
Forwarded 	TIME	02:00 PM
moved     	TIME	4:00
occurs    	DATE	a weekend
Forwarded 	TIME	12:07 PM
take      	DATE	Thursday
take      	TIME	3:00 - 
4:00
bs_stone@yahoo.com	TIME	09/26/2000 04:47:40 AM
am        	TIME	12 noon
occurs    	DATE	a weekend
With      	TIME	1 hour
occurs    	DATE	a weekend
relay     	DATE	37031
be        	DATE	Friday
position  	DATE	2002
are       	DATE	2003
are       	DATE	2004
booking   	DATE	Wednesday
cbpres@austin.rr.com	TIME	09/08/2000 05:21:49 AM
produce   	DATE	3 years
be        	DATE	less than 6 months
send      	DATE	Saturday delivery
reach     	DATE	the weekend
shut      	DATE	a week
the normal amount	DATE	August 28th
you       	DATE	the 2nd
payment   	DATE	October


KeyboardInterrupt: 